## 챕터4 텍스트와 바이트

* 문자, 코드 포인트, 바이트 표현
* 이진 시퀀스의 고유한 특징: bytes, bytearray, memoryview
* 전체 유니코드 및 레거시 문자셋에 대한 코덱
* 인코딩 에러를 피하고 다루기
* 텍스트 파일을 다룰 때의 모범 사례
* 기본 인코딩 및 표준 입출력 문제
* 정규화를 이용한 안전한 유니코드 텍스트 비교
* 정규화, 케이스 폴딩, 발음 구별 기호 강제 제거를 위한 유틸리티 함수
* locale과 PyUCA 라이브러리를 이용한 유니코드 텍스트의 적절한 정렬
* 유니코드 데이터베이스 안의 문자 메타데이터
* str과 bytes를 다루는 이중 모드 API


### 문자 문제
* 코드 포인트를 바이트로 변환하는 것을 **인코딩**, 바이트를 코드 포인트로 변환하는 것을 **디코딩**이라고 한다.

### 바이트에 대한 기본 지식
NOTE: s[0]==s[:1]이 되는 시퀀스형은 str이 유일하다. 그 외 모든 시퀀스의 경우, s[i]는 항목 하나를, s[i:i+1]은 안에 s[i] 항목을 가진 동일한 자료형의 시퀀스를 반환한다.

* 이진 시퀀스가 실제로 정수형의 시퀀스이지만, 리터럴 표기법을 보면 실제로는 아스키 텍스트가 들어가는 경우가 많다. 따라서 각 바이트 값에 따라 다음과 같이 세 가지 형태로 출력된다.
    - 화면에 출력 가능한 아스키 문자(공백에서 물결표(~)까지)는 아스키 문자 그대로 출력한다.
    - 탭, 개행 문자, 캐리지 리턴, 백슬래시(\)는 이스케이프 시퀀스(\t, \n, \r,\\)로 출력한다.
    - 그 외의 값은 널 바이트를 나타내는 \x00처럼 16진수 이스케이프 시퀀스로 출력한다.
    
### 기본 인코더/디코더

### 인코딩/디코딩 문제 이해하기
#### UnicodeEncodeError 처리하기
- 텍스트를 바이트로 변환할 때 문자가 대상 인코딩에 정의되어 있지 않으면, 인코딩 메서드나 함수의 errors 인수에 별도의 처리기를 지정하지 않는 한 UnicodeEncodeError가 발생한다.

#### UnicodeDecodeError 처리하기
- 이진 시퀀스를 텍스트로 변환할 때 정당한 문자로 변환할 때 정당한 문자로 변환할 수 없으면 UnicodeDecodeError가 발생한다.

### 제대로 비교하기 위해 유니코드 정규화하기
- 정규화 방식 NFC는 코드 포인트를 조합해서 가장 짧은 동일 문자열을 생성하는 반면, NFD는 조합된 문자를 기본 문자와 별도의 결합 문자로 분리한다.

#### 케이스 폴딩
- 본질적으로 케이스 폴딩은 모든 텍스트를 소문자로 변환하는 연산이며, 약간의 변환을 동반한다. 케이스 폴딩은 str.casefold() 메서드를 이용해서 수행한다.

#### 극단적인 '정규화': 발음 구별 기호 제거하기

In [5]:
# 결합 표시를 모두 제거하는 함수(sanitize.py 모듈)
import unicodedata
import string

def shave_marks(txt):
    """발음 구별 기호를 모두 제거한다"""
    norm_txt = unicodedata.normalize('NFD', txt) # 모든 문자를 기본 문자와 결합 표시로 분해한다.
    shaved = ''.join(c for c in norm_txt if not unicodedata.combining(c)) # 결합 표시를 모두 걸러낸다.
    return unicodedata.normalize('NFC', shaved) # 문자를 모두 재결합시킨다.

In [6]:
# 라틴 문자에서 결합 표시 기호를 제거하는 함수
def shave_marks_latin(txt):
    """라틴 기반 문자에서 발음 구별 기호를 모두 제거한다."""
    norm_txt = unicodedata.normalize('NFD', txt) # 모든 문자를 기반 문자와 결합 표시 기호로 분리한다.
    latin_base = False
    keepers = []
    for c in norm_txt:
        if unicodedata.combining(c) and latin_base: #기반 문자가 라틴 문자일 때 결합 표시 기호를 건너뛴다.
            continue # 라틴 기반 문자의 발음 구별 기호를 무시한다.
        keepers.append(c)
        # 결합 문자가 아니면, 이 문자를 새로운 기반 문자로 간주한다.
        if not unicodedata.combining(c): # 새로운 기반 문자를 찾아내고 라틴 문자인지 판단한다.
            latin_base = c in string.ascii_letters
    shaved = ''.join(keepers)
    return unicodedata.normalize('NFC', shaved) # 문자들을 모두 결합하고 NFC 방식으로 정규화한다.

### 유니코드 텍스트 정렬하기
#### 유니코드 대조 알고리즘을 이용한 정렬

### 유니코드 데이터베이스
- 유니코드 데이터베이스는 문자를 출력할 수 있는지, 문자인지, 십진수인지, 혹은 다른 수치형 기호인지 기록한다. str의 isidentifier(), isprintable(), isdecimal(), isnumeric() 메서드는 이 데이터베이스를 사용한다. 

### 이중 모드 str 및 bytes API
#### 정규 표현식에서의 str과 bytes
- bytes로 정규 표현식을 만들면 \d와 \w 같은 패턴은 아스키 문자만 매칭되지만, str로 이 패턴을 만들면 아스키 문자 이외에 유니코드 숫자나 문자도 매칭된다.

#### os 모듈 함수에서 str과 bytes
- GNU/리눅스 커널은 유니코드를 모른다. 따라서 실제 OS의 파일면은 어떠한 인코딩 체계에서도 올바르지 않은 바이트 시퀀스로 구성되어 있으며 str로 디코딩할 수 없다. 이 문제를 해결하기 위해 파일명이나 경로명을 받는 모든 os 모듈 함수는 str이나 bytes 형의 인수를 받는다. 이런 함수를 str 인수로 호출하면 인수는 sys.getfilesystemencoding() 함수에 의해 지정된 코덱을 이용해서 자동으로 변환되고, 운영 체계의 응답은 동일 코덱을 이용해서 디코딩한다. 그렇짐나 이렇게 처리할 수 없는 파일명을 다루거나 수정해야 할 때는 bytes 인수를 os 함수에 전달해서 bytes 반환값을 가져올 수 있다. 

- fsencode(파일명): '파일명'이 str 형이면 sys.getfilesystemencoding()이 반환한 코덱명을 이용해서 '파일명'을 bytes 형으로 인코딩한다. '파일명'이 bytes 형이면 변환하지 않고 그대로 반환한다.
- fsdecode(파일명): '파일명'이 bytes형이면 sys.getfilesystemencoding()이 반환한 코덱명을 이용해서 '파일명'을 str형으로 디코딩한다. '파일명'이 str 형이면 변환하지 않고 그대로 반환한다.